In [ ]:
!pip install langchain-openai
!pip install langgraph

In [ ]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict , Literal , Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage , HumanMessage

In [ ]:
generator_llm = ChatOpenAI(model_name="gpt-4o-mini")
evaluator_llm = ChatOpenAI(model_name="gpt-4o-mini")
optimization_llm = ChatOpenAI(model_name="gpt-4o")


In [ ]:
from pydantic import BaseModel , Field
class TweetEvaluation(BaseModel):
  evaluation: Literal["approved" , "need_improvement"] = Field(description="The evaluation of the tweet")
  feedback: str = Field(... , description="The feedback of the evaluation")

In [ ]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [ ]:
import operator
class TweetState(TypedDict):
  topic: str
  tweet: str
  evaluation: Literal["approved" , "need_improvement"]
  feedback: str
  iteration: int
  max_iteration: int

  tweet_hisotry: Annotated[list[str],operator.add]
  feedback_hisotry: Annotated[list[str],operator.add]

In [ ]:
graph = StateGraph(TweetState)

In [ ]:
def generate_tweet(state: TweetState):
  message = [
      SystemMessage(content="You are a funny and clever Twitter/X influencer."),
      HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
  ]

  responce = generator_llm.invoke(message).content

  return {'tweet': responce , 'tweet_hisotry':[responce]}

In [ ]:
def evaluate_tweet(state: TweetState):
  # prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?
2. Humor – Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness – Is it short, sharp, and scroll-stopping?
4. Virality Potential – Would people retweet or share it?
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: One paragraph explaining the strengths and weaknesses
""")
]
    response = structured_evaluator_llm.invoke(messages)
    return {'evaluation':response.evaluation, 'feedback': response.feedback , 'feedback_history':[response]}



In [ ]:
def optimze_tweet(state: TweetState):
  messages = [
      SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

  responce = optimization_llm.invoke(messages).content
  iteration = state['iteration'] + 1

  return {'tweet': responce, 'tweet_hisotry':[responce] , 'iteration': iteration}

In [ ]:
def route_evaluation(state: TweetState):
  if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
    return 'approved'
  else:
    return 'need_improvement'

In [ ]:
graph.add_node('generate',generate_tweet)
graph.add_node('evaluate',evaluate_tweet)
graph.add_node('optimize',optimze_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate' , route_evaluation , {'approved': END , 'need_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')


In [ ]:
workflow = graph.compile()

In [ ]:
intial_state = {
    "topic": "dfgfghdfgh",
    "iteration": 1,
    "max_iteration": 3
}

workflow.invoke(intial_state)

{'topic': 'dfgfghdfgh',
 'tweet': 'Saw someone type "dfgfghdfgh" — not keyboard smashing, it’s modern art. 🤦\u200d♂️ We\'ve all been there. #CodingInPanicMode #KeyboardFury',
 'evaluation': 'approved',
 'feedback': "This tweet is a refreshing take on the age-old phenomenon of keyboard smashing, framing it as 'modern art' which adds an original twist. The humor lies in the relatability of the situation, and the emoji adds a subtle comedic touch. It’s concise and maintains a punchy format that's easily scroll-stopping. With hashtags that appeal to a niche audience, it has good virality potential, likely resonating well within coding communities. Overall, it strikes a solid balance between humor, originality, and format.",
 'iteration': 2,
 'max_iteration': 3,
 'tweet_hisotry': ['Just saw someone type "dfgfghdfgh" and I now understand the struggle. That\'s not keyboard smashing; that\'s a cry for help! We’ve all been there, but I like to pretend I’m just typing in my “secret language.” 🤷\